In [3]:
import numpy as np
import pandas as pd

In [4]:
!pip install pyodbc pandas


In [5]:
!pip install sqlalchemy pyodbc pandas


In [3]:
from sqlalchemy import create_engine
import pandas as pd

# Define the connection string
connection_string = (
    "mssql+pyodbc://DESKTOP-45PMML3/AdventureWorks2022"
    "?driver=SQL+Server"
)

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

try:
    # Define SQL query
    query = "SELECT TOP 10 * FROM [HumanResources].[Department]"

    # Execute query and Load into DataFrame
    df = pd.read_sql(query, engine)

    # Show the results
    print(df.head())

except Exception as e:
    print(f"Error: {e}")


   DepartmentID         Name                 GroupName ModifiedDate
0             1  Engineering  Research and Development   2008-04-30
1             2  Tool Design  Research and Development   2008-04-30
2             3        Sales       Sales and Marketing   2008-04-30
3             4    Marketing       Sales and Marketing   2008-04-30
4             5   Purchasing      Inventory Management   2008-04-30


In [6]:
# Perform a merge based on BusinessEntityID (equivalent to JOIN in SQL)
merged_df = pd.merge(employee_df[['BusinessEntityID']], 
                     person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                     on='BusinessEntityID')

# Rename the BusinessEntityID column to EmployeeID
merged_df = merged_df.rename(columns={'BusinessEntityID': 'EmployeeID'})

# Display the result
print(merged_df)


     EmployeeID FirstName           LastName
0             1       Ken            Sánchez
1             2     Terri              Duffy
2             3   Roberto         Tamburello
3             4       Rob            Walters
4             5      Gail           Erickson
..          ...       ...                ...
285         286      Lynn           Tsoflias
286         287       Amy            Alberts
287         288    Rachel             Valdez
288         289       Jae                Pak
289         290    Ranjit  Varkey Chudukatil

[290 rows x 3 columns]


In [8]:
# 2. Sort the Employee DataFrame by LastName in ascending order
sorted_employee_df = employee_df.sort_values(by='LastName')
print(sorted_employee_df)

     EmployeeID FirstName     LastName
59          285      Syed        Abbas
251          38       Kim  Abercrombie
142         211     Hazem     Abolrous
239         121     Pilar     Ackerman
240          67       Jay        Adams
..          ...       ...          ...
248          99      Nuan           Yu
212         129      Gary       Yukish
39          173    Eugene  Zabokritski
56           91  Kimberly    Zimmerman
234          64   Michael     Zwilling

[290 rows x 3 columns]


In [5]:

job_titles_df = employee_df[['JobTitle']].drop_duplicates()
print(job_titles_df)


                          JobTitle
0          Chief Executive Officer
1    Vice President of Engineering
2              Engineering Manager
3             Senior Tool Designer
4                  Design Engineer
..                             ...
272        Vice President of Sales
273   North American Sales Manager
274           Sales Representative
284          Pacific Sales Manager
286         European Sales Manager

[67 rows x 1 columns]


In [4]:
# List of all tables in HumanResources schema and Person.Person table
import pandas as pd
tables = [
    "HumanResources.Employee",
    "HumanResources.Department",
    "HumanResources.EmployeeDepartmentHistory",
    "HumanResources.EmployeePayHistory",
    "HumanResources.JobCandidate",
    "HumanResources.Shift",
    "Person.Person"
]

# Load each table into a dictionary of DataFrames
dfs = {}
for table in tables:
    query = f"SELECT * FROM {table}"
    dfs[table.split('.')[-1]] = pd.read_sql(query, engine)

# Access each table using the table name
employee_df = dfs['Employee']
department_df = dfs['Department']
employee_department_history_df = dfs['EmployeeDepartmentHistory']
employee_pay_history_df = dfs['EmployeePayHistory']
job_candidate_df = dfs['JobCandidate']
shift_df = dfs['Shift']
person_df = dfs['Person']

# Example: Print Employee DataFrame
print(employee_df)


     BusinessEntityID NationalIDNumber                   LoginID  \
0                   1        295847284      adventure-works\ken0   
1                   2        245797967    adventure-works\terri0   
2                   3        509647174  adventure-works\roberto0   
3                   4        112457891      adventure-works\rob0   
4                   5        695256908     adventure-works\gail0   
..                ...              ...                       ...   
285               286        758596752     adventure-works\lynn0   
286               287        982310417      adventure-works\amy0   
287               288        954276278   adventure-works\rachel0   
288               289        668991357      adventure-works\jae0   
289               290        134219713   adventure-works\ranjit0   

    OrganizationNode  OrganizationLevel                       JobTitle  \
0               None                NaN        Chief Executive Officer   
1               b'X'               

In [7]:
# 4. Filter employees with JobTitle containing 'Manager'

merged_df = pd.merge(employee_df[['BusinessEntityID', 'JobTitle']], 
                     person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                     on='BusinessEntityID')

# Filter rows where 'JobTitle' contains 'Manager'
manager_df = merged_df[merged_df['JobTitle'].str.contains('Manager', case=False, na=False)]

# Rename 'BusinessEntityID' to 'EmployeeID'
manager_df = manager_df.rename(columns={'BusinessEntityID': 'EmployeeID'})

# Display the result
print(manager_df)


     EmployeeID                          JobTitle  FirstName  \
2             3               Engineering Manager    Roberto   
6             7  Research and Development Manager      Dylan   
9            10  Research and Development Manager    Michael   
15           16                 Marketing Manager      David   
25           26        Production Control Manager      Peter   
210         211         Quality Assurance Manager      Hazem   
216         217          Document Control Manager     Zainal   
226         227                Facilities Manager       Gary   
234         235           Human Resources Manager      Paula   
240         241                  Accounts Manager      David   
248         249                   Finance Manager      Wendy   
249         250                Purchasing Manager     Sheela   
262         263      Information Services Manager       Jean   
263         264                   Network Manager  Stephanie   
273         274      North American Sale

In [8]:
# 5. Count the total number of employees in the Employee DataFrame

# Get the total number of employees
total_employees = len(employee_df)

# Convert it to a DataFrame for consistent format
total_employees_df = pd.DataFrame({'TotalEmployees': [total_employees]})

# Display the result
print(total_employees_df)


   TotalEmployees
0             290


In [9]:
# 6. Find employees who are currently on leave.				
# Filter the rows where 'CurrentFlag' is 0 (on leave)
onleave_df = employee_df[employee_df['CurrentFlag'] == 0][['BusinessEntityID']]

# Display the result
print(onleave_df)


Empty DataFrame
Columns: [BusinessEntityID]
Index: []


In [10]:
# 7. Check for any missing hire dates in the Employee DataFrame

# Merge Employee, Person, and EmployeePayHistory DataFrames on 'BusinessEntityID'
merged_df = pd.merge(employee_df[['BusinessEntityID']], 
                     person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                     on='BusinessEntityID')
merged_df = pd.merge(merged_df, 
                     employee_pay_history_df[['BusinessEntityID', 'RateChangeDate']], 
                     on='BusinessEntityID')

# Filter rows where 'RateChangeDate' is missing (NaN)
missing_hire_dates_df = merged_df[merged_df['RateChangeDate'].isna()]

# Rename 'BusinessEntityID' to 'EmployeeID'
missing_hire_dates_df = missing_hire_dates_df.rename(columns={'BusinessEntityID': 'EmployeeID'})

# Display the result
print(missing_hire_dates_df)


Empty DataFrame
Columns: [EmployeeID, FirstName, LastName, RateChangeDate]
Index: []


In [11]:
from datetime import datetime, timedelta

# 8. Select employees who are eligible for retirement

from datetime import datetime

# Merge Employee and Person DataFrames on 'BusinessEntityID'
merged_df = pd.merge(employee_df[['BusinessEntityID', 'BirthDate']], 
                     person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                     on='BusinessEntityID')

# Calculate age
current_year = datetime.now().year
merged_df['Age'] = current_year - pd.to_datetime(merged_df['BirthDate']).dt.year

# Filter rows where age is 60 or more
retirement_eligible_df = merged_df[merged_df['Age'] >= 60]

# Display the result
print(retirement_eligible_df[['BusinessEntityID', 'FirstName', 'LastName', 'Age']])


     BusinessEntityID FirstName        LastName  Age
4                   5      Gail        Erickson   72
5                   6    Jossef        Goldberg   65
11                 12   Thierry          D'Hers   65
14                 15    Sharon       Salavaria   63
22                 23      Mary          Gibson   62
26                 27        Jo           Brown   68
39                 40    JoLynn          Dobney   68
47                 48      Ruth      Ellerbrock   68
48                 49     Barry         Johnson   68
49                 50    Sidney            Higa   68
50                 51   Jeffrey            Ford   68
51                 52     Doris         Hartwig   68
52                 53     Diane           Glimp   68
54                 55    Taylor         Maxwell   68
61                 62      John        Campbell   68
62                 63    Maciej           Dusza   69
83                 84     Frank        Martinez   72
139               140  Prasanna  Samarawickram

In [12]:
# 9. Calculate the total number of employees in each department

# Merge Employee, EmployeeDepartmentHistory, and Department DataFrames on 'BusinessEntityID' and 'DepartmentID'
merged_df = pd.merge(employee_df[['BusinessEntityID']], 
                     employee_department_history_df[['BusinessEntityID', 'DepartmentID']], 
                     on='BusinessEntityID')
merged_df = pd.merge(merged_df, 
                     department_df[['DepartmentID', 'Name']], 
                     on='DepartmentID')

# Group by 'DepartmentID' and 'Name', then count employees
employees_per_department_df = merged_df.groupby(['DepartmentID', 'Name']).size().reset_index(name='TotalEmployees')

# Display the result
print(employees_per_department_df)


    DepartmentID                        Name  TotalEmployees
0              1                 Engineering               7
1              2                 Tool Design               4
2              3                       Sales              18
3              4                   Marketing              10
4              5                  Purchasing              13
5              6    Research and Development               4
6              7                  Production             180
7              8          Production Control               6
8              9             Human Resources               6
9             10                     Finance              11
10            11        Information Services              10
11            12            Document Control               5
12            13           Quality Assurance               7
13            14  Facilities and Maintenance               7
14            15      Shipping and Receiving               6
15            16        

In [13]:
# 10. Merge the Employee and Department DataFrames on DepartmentID
# Merge Employee, Person, EmployeeDepartmentHistory, and Department DataFrames
employee_department_df = pd.merge(employee_df[['BusinessEntityID']], 
                                  person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                                  on='BusinessEntityID')
employee_department_df = pd.merge(employee_department_df, 
                                  employee_department_history_df[['BusinessEntityID', 'DepartmentID']], 
                                  on='BusinessEntityID')
employee_department_df = pd.merge(employee_department_df, 
                                  department_df[['DepartmentID', 'Name']], 
                                  on='DepartmentID')

# Rename Department Name column
employee_department_df = employee_department_df.rename(columns={'Name': 'DepartmentName'})

# Display the result
print(employee_department_df)


     BusinessEntityID FirstName           LastName  DepartmentID  \
0                   1       Ken            Sánchez            16   
1                   2     Terri              Duffy             1   
2                   3   Roberto         Tamburello             1   
3                   4       Rob            Walters             1   
4                   4       Rob            Walters             2   
..                ...       ...                ...           ...   
291               286      Lynn           Tsoflias             3   
292               287       Amy            Alberts             3   
293               288    Rachel             Valdez             3   
294               289       Jae                Pak             3   
295               290    Ranjit  Varkey Chudukatil             3   

    DepartmentName  
0        Executive  
1      Engineering  
2      Engineering  
3      Engineering  
4      Tool Design  
..             ...  
291          Sales  
292          Sa

In [29]:

# 11. Create an intermediate DataFrame to calculate the number of employees hired in the last 12 months
import pandas as pd
from datetime import datetime, timedelta


# Calculate the date 12 months ago
twelve_months_ago = datetime.now() - timedelta(days=365)

# Convert 'StartDate' column to datetime if it's not already
employee_department_history_df['StartDate'] = pd.to_datetime(employee_department_history_df['StartDate'])

# Filter EmployeeDepartmentHistory based on 'StartDate' being within the last 12 months
recently_hired_df = employee_department_history_df[employee_department_history_df['StartDate'] >= twelve_months_ago]

# Get the count of distinct employees
number_of_people = len(recently_hired_df['BusinessEntityID'].unique())

# Create DataFrame for the result
recently_hired_df = pd.DataFrame({'NumberOfPeople': [number_of_people]})

# Display the result
print(recently_hired_df)



   NumberOfPeople
0               0


In [16]:
# 12. Filter departments with more than 50 employees
# Group by DepartmentID and Department Name, and count employees
employees_per_department_df = employee_department_history_df.groupby('DepartmentID').size().reset_index(name='TotalEmployees')

# Merge to get department names
employees_per_department_df = pd.merge(employees_per_department_df, department_df[['DepartmentID', 'Name']], on='DepartmentID')

# Filter departments with more than 50 employees
large_departments_df = employees_per_department_df[employees_per_department_df['TotalEmployees'] > 50]

# Display the result
print(large_departments_df)


   DepartmentID  TotalEmployees        Name
6             7             180  Production


In [17]:
# 13. Identify the average tenure of employees in each department
from datetime import datetime

# Calculate current date
current_date = datetime.now()

# Merge to get necessary details
merged_df = pd.merge(employee_df[['BusinessEntityID', 'HireDate']], 
                     employee_department_history_df[['BusinessEntityID', 'DepartmentID']], 
                     on='BusinessEntityID')
merged_df = pd.merge(merged_df, department_df[['DepartmentID', 'Name']], on='DepartmentID')

# Calculate tenure for each employee
merged_df['TenureYears'] = (current_date.year - pd.to_datetime(merged_df['HireDate']).dt.year)

# Group by DepartmentID and calculate the average tenure
average_tenure_df = merged_df.groupby(['DepartmentID', 'Name'])['TenureYears'].mean().reset_index(name='AvgTenureYears')

# Display the result
print(average_tenure_df)


    DepartmentID                        Name  AvgTenureYears
0              1                 Engineering       15.571429
1              2                 Tool Design       15.500000
2              3                       Sales       12.444444
3              4                   Marketing       14.700000
4              5                  Purchasing       14.384615
5              6    Research and Development       15.250000
6              7                  Production       15.194444
7              8          Production Control       15.333333
8              9             Human Resources       15.500000
9             10                     Finance       15.181818
10            11        Information Services       15.300000
11            12            Document Control       15.200000
12            13           Quality Assurance       14.857143
13            14  Facilities and Maintenance       14.571429
14            15      Shipping and Receiving       15.500000
15            16        

In [18]:
# 14. Calculate the turnover rate for the company
# Count employees who left 
employees_left = employee_department_history_df[employee_department_history_df['EndDate'].notna()]['BusinessEntityID'].nunique()

# Total number of employees
total_employees = employee_df['BusinessEntityID'].nunique()

# Calculate turnover rate
turnover_rate = (employees_left / total_employees) * 100

# Create DataFrame for the result
turnover_df = pd.DataFrame({'EmployeesLeft': [employees_left], 'TotalEmployees': [total_employees], 'TurnoverRate': [turnover_rate]})

# Display the result
print(turnover_df)


   EmployeesLeft  TotalEmployees  TurnoverRate
0              5             290      1.724138


In [19]:
# 15. Determine the number of employees in each age group
# Assuming 'employee_df' contains 'BusinessEntityID' and 'BirthDate' columns

from datetime import datetime

# Calculate the age of employees
current_year = datetime.now().year
employee_df['Age'] = current_year - pd.to_datetime(employee_df['BirthDate']).dt.year

# Define age groups
def age_group(age):
    if age < 30:
        return 'Under 30'
    elif 30 <= age <= 50:
        return '30-50'
    else:
        return 'Over 50'

# Apply the age_group function to create the 'AgeGroup' column
employee_df['AgeGroup'] = employee_df['Age'].apply(age_group)

# Group by 'AgeGroup' and count employees
age_group_df = employee_df.groupby('AgeGroup').size().reset_index(name='TotalEmployees')

# Display the result
print(age_group_df)


  AgeGroup  TotalEmployees
0    30-50             215
1  Over 50              75


In [20]:
#16. Filter employees who have not completed mandatory training.				
# Assuming 'person_df', 'employee_df', and 'employee_pay_history_df' contain necessary columns

# Merge Person, Employee, and EmployeePayHistory DataFrames
merged_df = pd.merge(employee_df[['BusinessEntityID']], 
                     person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                     on='BusinessEntityID')
merged_df = pd.merge(merged_df, 
                     employee_pay_history_df[['BusinessEntityID', 'PayFrequency']], 
                     on='BusinessEntityID')

# Filter rows where PayFrequency is 2
employee_temp_df = merged_df[merged_df['PayFrequency'] == 2]

# Display the result
print(employee_temp_df[['FirstName', 'LastName', 'BusinessEntityID', 'PayFrequency']])


    FirstName           LastName  BusinessEntityID  PayFrequency
0         Ken            Sánchez                 1             2
1       Terri              Duffy                 2             2
2     Roberto         Tamburello                 3             2
3         Rob            Walters                 4             2
4         Rob            Walters                 4             2
..        ...                ...               ...           ...
311      Lynn           Tsoflias               286             2
312       Amy            Alberts               287             2
313    Rachel             Valdez               288             2
314       Jae                Pak               289             2
315    Ranjit  Varkey Chudukatil               290             2

[136 rows x 4 columns]


In [21]:
# 17. Rank employees by hire date
# Assuming 'employee_df' contains 'BusinessEntityID' and 'HireDate' columns

# Convert 'HireDate' to datetime if not already in that format
employee_df['HireDate'] = pd.to_datetime(employee_df['HireDate'])

# Rank employees by 'HireDate'
employee_df['Rank'] = employee_df['HireDate'].rank(ascending=True)

# Sort by rank
employee_hire_df = employee_df[['BusinessEntityID', 'HireDate', 'Rank']].sort_values('Rank')

# Display the result
print(employee_hire_df)


     BusinessEntityID   HireDate   Rank
27                 28 2006-06-30    1.0
16                 17 2007-01-26    2.0
2                   3 2007-11-11    3.0
3                   4 2007-12-05    4.0
11                 12 2007-12-11    5.0
..                ...        ...    ...
289               290 2012-05-30  285.5
283               284 2012-09-30  287.0
284               285 2013-03-14  288.0
285               286 2013-05-30  289.5
287               288 2013-05-30  289.5

[290 rows x 3 columns]


In [22]:
# 18. Top 5 longest-serving employees
# Assuming 'employee_hire_df' contains 'BusinessEntityID' and 'HireDate' columns

# Sort employees by 'HireDate' in ascending order to get the longest-serving employees
longest_serving_df = employee_hire_df.sort_values('HireDate').head(5)

# Display the result
print(longest_serving_df)



    BusinessEntityID   HireDate  Rank
27                28 2006-06-30   1.0
16                17 2007-01-26   2.0
2                  3 2007-11-11   3.0
3                  4 2007-12-05   4.0
11                12 2007-12-11   5.0


In [23]:
# 19. Function to retrieve employee details based on DepartmentID and HireDate


def get_employee_by_dept_and_hiredate(department_id, hire_date):
    # Merge Employee, Person, EmployeeDepartmentHistory, and Department DataFrames
    merged_df = pd.merge(employee_df[['BusinessEntityID', 'HireDate']], 
                         person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                         on='BusinessEntityID')
    merged_df = pd.merge(merged_df, 
                         employee_department_history_df[['BusinessEntityID', 'DepartmentID']], 
                         on='BusinessEntityID')
    merged_df = pd.merge(merged_df, 
                         department_df[['DepartmentID', 'Name']], 
                         left_on='DepartmentID', right_on='DepartmentID')
    
    # Filter based on DepartmentID and HireDate
    result_df = merged_df[(merged_df['DepartmentID'] == department_id) & (merged_df['HireDate'] == hire_date)]
    
    return result_df[['BusinessEntityID', 'FirstName', 'LastName', 'Name', 'HireDate']]

dept_employee_df = get_employee_by_dept_and_hiredate(16, '2009-01-14')
print(dept_employee_df)



   BusinessEntityID FirstName LastName       Name   HireDate
0                 1       Ken  Sánchez  Executive 2009-01-14


In [24]:
# 20. Days since last promotion (assuming last promotion is the most recent RateChangeDate)

# Convert 'RateChangeDate' to datetime if not already
employee_pay_history_df['RateChangeDate'] = pd.to_datetime(employee_pay_history_df['RateChangeDate'])

# Calculate days since the last promotion
most_recent_promotion_df = employee_pay_history_df.groupby('BusinessEntityID')['RateChangeDate'].max().reset_index()
most_recent_promotion_df['DaysSinceLastPromotion'] = (pd.Timestamp.now() - most_recent_promotion_df['RateChangeDate']).dt.days

# Display the result
print(most_recent_promotion_df)


     BusinessEntityID RateChangeDate  DaysSinceLastPromotion
0                   1     2009-01-14                    5726
1                   2     2008-01-31                    6075
2                   3     2007-11-11                    6156
3                   4     2011-12-15                    4661
4                   5     2008-01-06                    6100
..                ...            ...                     ...
285               286     2013-05-30                    4129
286               287     2012-04-16                    4538
287               288     2013-05-30                    4129
288               289     2012-05-30                    4494
289               290     2012-05-30                    4494

[290 rows x 3 columns]


In [25]:
#21. Analyze the effectiveness of employee development programs.						


# Group by 'EmailPromotion' and count the total number of people in each group
email_promotion_df = person_df.groupby('EmailPromotion').size().reset_index(name='TotalPeople')

# Display the result
print(email_promotion_df)



   EmailPromotion  TotalPeople
0               0        11158
1               1         5044
2               2         3770


In [26]:
# Query to find employees with the highest number of shifts
# Merge EmployeeDepartmentHistory, Employee, and Person DataFrames
merged_shift_df = pd.merge(employee_department_history_df[['BusinessEntityID', 'ShiftID']], 
                           employee_df[['BusinessEntityID']], on='BusinessEntityID')
merged_shift_df = pd.merge(merged_shift_df, 
                           person_df[['BusinessEntityID', 'FirstName', 'LastName']], 
                           on='BusinessEntityID')

# Count the number of shifts per employee and sort by 'ShiftCount'
shift_count_df = merged_shift_df.groupby(['BusinessEntityID', 'FirstName', 'LastName']).size().reset_index(name='ShiftCount')
top_shift_employees_df = shift_count_df.sort_values('ShiftCount', ascending=False)

# Display the result
print(top_shift_employees_df)

     BusinessEntityID  FirstName           LastName  ShiftCount
249               250     Sheela               Word           3
3                   4        Rob            Walters           2
233               234      Laura             Norman           2
223               224    William               Vong           2
15                 16      David            Bradley           2
..                ...        ...                ...         ...
97                 98     Sameer             Tejani           1
96                 97     Mandar             Samant           1
95                 96  Elizabeth             Keyser           1
94                 95        Jim          Scardelis           1
289               290     Ranjit  Varkey Chudukatil           1

[290 rows x 4 columns]


In [27]:
# 23

# Merge EmployeeDepartmentHistory, Employee, and Shift DataFrames
merged_shift_df = pd.merge(employee_department_history_df[['BusinessEntityID', 'ShiftID']], 
                           employee_df[['BusinessEntityID']], 
                           on='BusinessEntityID')
merged_shift_df = pd.merge(merged_shift_df, 
                           shift_df[['ShiftID', 'StartTime', 'EndTime']], 
                           on='ShiftID')

# Convert 'StartTime' and 'EndTime' to datetime if not already
merged_shift_df['StartTime'] = pd.to_datetime(merged_shift_df['StartTime'])
merged_shift_df['EndTime'] = pd.to_datetime(merged_shift_df['EndTime'])

# Calculate Shift Duration in hours
merged_shift_df['ShiftDuration'] = abs((merged_shift_df['EndTime'] - merged_shift_df['StartTime']).dt.total_seconds() / 3600)

# Select required columns
compliance_df = merged_shift_df[['BusinessEntityID', 'ShiftDuration']]

# Display the result
print(compliance_df)



     BusinessEntityID  ShiftDuration
0                   1            8.0
1                   2            8.0
2                   3            8.0
3                   4            8.0
4                   4            8.0
..                ...            ...
291               286            8.0
292               287            8.0
293               288            8.0
294               289            8.0
295               290            8.0

[296 rows x 2 columns]


C:\Users\HP\AppData\Local\Temp\ipykernel_14196\940224241.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_shift_df['StartTime'] = pd.to_datetime(merged_shift_df['StartTime'])
C:\Users\HP\AppData\Local\Temp\ipykernel_14196\940224241.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_shift_df['EndTime'] = pd.to_datetime(merged_shift_df['EndTime'])


In [28]:
#24. Determine the distribution of employee performance ratings across departments.				

# Merge Person, Employee, EmployeeDepartmentHistory, and Department DataFrames
merged_df = pd.merge(person_df[['BusinessEntityID', 'EmailPromotion']], 
                     employee_df[['BusinessEntityID']], 
                     on='BusinessEntityID')
merged_df = pd.merge(merged_df, 
                     employee_department_history_df[['BusinessEntityID', 'DepartmentID']], 
                     on='BusinessEntityID')
merged_df = pd.merge(merged_df, 
                     department_df[['DepartmentID', 'Name']], 
                     left_on='DepartmentID', right_on='DepartmentID')

# Group by DepartmentName and sum EmailPromotion
shift_email_distribution_df = merged_df.groupby('Name').agg({'EmailPromotion': 'sum'}).reset_index()
shift_email_distribution_df.rename(columns={'Name': 'DepartmentName', 'EmailPromotion': 'TotalPromotionEmails'}, inplace=True)

# Rank the departments by TotalPromotionEmails
shift_email_distribution_df['Rank'] = shift_email_distribution_df['TotalPromotionEmails'].rank(ascending=False)

# Display the result sorted by rank
print(shift_email_distribution_df.sort_values('Rank'))


                DepartmentName  TotalPromotionEmails  Rank
8                   Production                   129   1.0
7                    Marketing                    10   2.5
13                       Sales                    10   2.5
10                  Purchasing                     8   4.0
4                      Finance                     6   5.5
6         Information Services                     6   5.5
1                  Engineering                     5   7.0
5              Human Resources                     4  10.0
11           Quality Assurance                     4  10.0
12    Research and Development                     4  10.0
14      Shipping and Receiving                     4  10.0
15                 Tool Design                     4  10.0
0             Document Control                     2  13.5
9           Production Control                     2  13.5
2                    Executive                     1  15.0
3   Facilities and Maintenance                     0  16